In [56]:
# import pandas as pd

# # Read the Excel file
# file_path = 'test_cases.xlsx'  # Update this path to the actual path of your Excel file
# df = pd.read_excel(file_path, header=None)

# # Initialize variables
# use_case_scenarios = {}
# current_use_case = None

# # Iterate through the rows of the DataFrame
# for index, row in df.iterrows():
#     test_scenario = row[0]
#     if pd.isna(test_scenario):
#         continue
#     if test_scenario.endswith('.txt'):

#         current_use_case = test_scenario.replace('.txt','')
#         use_case_scenarios[current_use_case] = []
#     elif current_use_case:
#         use_case_scenarios[current_use_case].append(test_scenario)

# # Convert the dictionary to a 2D array
# result_array = [[use_case, scenarios] for use_case, scenarios in use_case_scenarios.items()]

# # Print the result
# for item in result_array:
#     print(item)

In [57]:
import pandas as pd

# Read the Excel file
file_path = r"D:\hotel-ts-baseline.xlsx" # Update this path to the actual path of your Excel file
df = pd.read_excel(file_path, header=None)

# Initialize variables
use_case_scenarios = {}
current_use_case = None

# Iterate through the rows of the DataFrame
for index, row in df.iterrows():
    test_scenario = row[0]
    valid = row[1]
    if pd.isna(test_scenario):
        continue
    if test_scenario.endswith('.txt'):
        current_use_case = test_scenario.replace('.txt','')
        use_case_scenarios[current_use_case] = []
    elif current_use_case and valid == 'x':
        use_case_scenarios[current_use_case].append(test_scenario)

# # Convert the dictionary to a 2D array
# result_array = [[use_case, scenarios] for use_case, scenarios in use_case_scenarios.items()]

# # Print the result
# for item in result_array:
#     print(item)
file_list = []
print(use_case_scenarios)
for key, value in use_case_scenarios.items():
    for scenario in value:

        file_list.append(key+'-'+scenario+'.txt')
print(file_list)

{'Add Booking': ['1. SuccessfulBookingAddition', '2. BookingWithoutLogin', '5. CalculateTotalAmount', '8. AddBookingWithoutFillingFields', '13. CalculateAmountWithoutFillingFields', '14. AddBookingWithoutCalculatingTotalAmount', '18. AddBookingWithPartialFieldsFilled', '19. LogoutAndTryToAddBooking'], 'Add Employee': ['1. Test Scenario: Successful Employee Addition', '2. Test Scenario: Manager Not Logged In', '3. Test Scenario: Mandatory Fields Left Blank', '4. Test Scenario: Invalid Field Entry', '7. Test Scenario: Refresh Page During Employee Addition'], 'Add Room': ['1. AddRoom_SuccessfulAddition', '2. AddRoom_InvalidFieldInput', '3. AddRoom_RoomAlreadyExists', '4. AddRoom_MissingMandatoryFields', '11. AddRoom_AccessControl_ManagerOnly'], 'Check in': ['1. Test Scenario: Verify successful check-in with valid booking ID', '2. Test Scenario: Verify system behavior with invalid booking ID', '3. Test Scenario: Verify system response when no booking ID is entered', '9. Test Scenario: Veri

In [58]:
import re
import pandas as pd
import os
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."

def split_before_and_after_hyphen(file_name):
    parts = file_name.split('-', 1)
    if len(parts) == 2:
        before_hyphen, after_hyphen = parts
    else:
        before_hyphen, after_hyphen = file_name, ""
    return before_hyphen.strip(), after_hyphen.strip()

# Function to parse the test cases from the text file
def parse_test_cases(file_name):
    # with open(file_name, 'r') as file:
    #     content = file.read()
    content = read_file_content(file_name)
    base_name = os.path.basename(file_name)

    base_nam = os.path.splitext(base_name)[0]

    # Extract use case and test scenario names from the file name
    use_case_name, test_scenario_name = split_before_and_after_hyphen(base_nam)
    
    # Regular expression to extract test cases
    test_case_pattern = re.compile(
        r"Test Case: (.*?)\ntestCaseName: (.*?)\nobjective: (.*?)\ntestSteps:\n(.*?)\nexpectedResult: (.*?)\n",
        re.DOTALL
    )
    
    matches = test_case_pattern.findall(content)
    
    test_cases = []
    for match in matches:
        test_case_name = match[1].strip()
        objective = match[2].strip()
        test_steps = match[3].strip().replace('\n', ' ')
        expected_result = match[4].strip()
        
        test_cases.append({
            'Use Case Name': use_case_name,
            'Test Scenario Name': test_scenario_name,
            'Test Case Name': test_case_name,
            'Objective': objective,
            'Test Steps': test_steps,
            'Expected Result': expected_result
        })
    
    return test_cases

# Function to write the parsed test cases to an Excel file
def write_to_excel(test_cases, output_file):
    df = pd.DataFrame(test_cases)
    df.to_excel(output_file, index=False)

all_test_cases = []

# File names
# input_file = 'use_case-test_scenario.txt'
output_file = 'test_cases.xlsx'
folder_path = r'D:\Dissertation-GPT\evaluate\baseline 2\hotel\tc'
# Parse the test cases and write to Excel
# file_list = os.listdir(folder_path)


def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

# Function to sort filenames naturally by the extracted number
def natural_sort_key(filename):
    # Split the filename into parts: prefix and number
    parts = re.split(r'(\d+)', filename)
    # Convert numerical parts to integers for proper numeric sorting
    parts[1::2] = map(int, parts[1::2])
    return parts

# Sort filenames using the natural sort key
sorted_files = sorted(file_list, key=natural_sort_key)

for i in sorted_files: 
    print(i)
for filename in sorted_files:
    file_path = os.path.join(folder_path, filename)
    print(file_path)
    if os.path.isfile(file_path):
        test_cases = parse_test_cases(file_path)
        # print(test_cases)
        test_cases = parse_test_cases(file_path)
        all_test_cases.extend(test_cases)
write_to_excel(all_test_cases, output_file)
print(all_test_cases)



Add Booking-1. SuccessfulBookingAddition.txt
Add Booking-2. BookingWithoutLogin.txt
Add Booking-5. CalculateTotalAmount.txt
Add Booking-8. AddBookingWithoutFillingFields.txt
Add Booking-13. CalculateAmountWithoutFillingFields.txt
Add Booking-14. AddBookingWithoutCalculatingTotalAmount.txt
Add Booking-18. AddBookingWithPartialFieldsFilled.txt
Add Booking-19. LogoutAndTryToAddBooking.txt
Add Employee-1. Test Scenario: Successful Employee Addition.txt
Add Employee-2. Test Scenario: Manager Not Logged In.txt
Add Employee-3. Test Scenario: Mandatory Fields Left Blank.txt
Add Employee-4. Test Scenario: Invalid Field Entry.txt
Add Employee-7. Test Scenario: Refresh Page During Employee Addition.txt
Add Room-1. AddRoom_SuccessfulAddition.txt
Add Room-2. AddRoom_InvalidFieldInput.txt
Add Room-3. AddRoom_RoomAlreadyExists.txt
Add Room-4. AddRoom_MissingMandatoryFields.txt
Add Room-11. AddRoom_AccessControl_ManagerOnly.txt
Check in-1. Test Scenario: Verify successful check-in with valid booking I